In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from MODULES.encoders_decoders import *
import torch

In [4]:
from MODULES.vae_model import *
from MODULES.utilities_ml import process_one_epoch
from MODULES.utilities_visualization import show_batch
import torch.nn.functional as F
from MODULES.utilities_ml import ConditionalRandomCrop, SpecialDataSet, process_one_epoch


# Check versions
from platform import python_version
print("python_version() ---> ", python_version())
print("torch.__version__ --> ", torch.__version__)

python_version() --->  3.8.2
torch.__version__ -->  1.6.0


In [23]:
multi_mnist_data_train = "/Users/ldalessi/DAPI_unsupervised/spacetx-research/dataset_multiMNIST/multi_mnist_train.pt"
multi_mnist_data_test = "/Users/ldalessi/DAPI_unsupervised/spacetx-research/dataset_multiMNIST/multi_mnist_test.pt"

params = load_json_as_dict("./ML_parameters.json")
img_train, seg_mask_train, count_train  = load_obj(multi_mnist_data_train)
img_test, seg_mask_test, count_test  = load_obj(multi_mnist_data_test)

BATCH_SIZE = params["simulation"]["batch_size"]
SIZE_CROPS = params["input_image"]["size_raw_image"]
N_TEST = params["simulation"]["N_test"]
N_TRAIN = params["simulation"]["N_train"]

train_loader = SpecialDataSet(img=img_train, 
                              roi_mask=None, 
                              seg_mask=seg_mask_train, 
                              labels=count_train, 
                              batch_size=4, #BATCH_SIZE , 
                              drop_last=False,
                              shuffle=True)

test_loader = SpecialDataSet(img=img_test, 
                             roi_mask=None, 
                             seg_mask=seg_mask_test, 
                             labels=count_test, 
                             batch_size=4, #BATCH_SIZE , 
                             drop_last=False,
                             shuffle=False)

test_batch_example_fig = test_loader.check_batch()
train_batch_example_fig = train_loader.check_batch()

Dataset lenght: 10
img.shape torch.Size([10, 1, 80, 80])
img.dtype torch.float32
img.device cpu
MINIBATCH: img.shapes seg.shape labels.shape, index.shape -> torch.Size([8, 1, 80, 80]) torch.Size([8, 1, 80, 80]) torch.Size([8]) torch.Size([8])
MINIBATCH: min and max of minibatch tensor(0.) tensor(1.)
Dataset lenght: 10
img.shape torch.Size([10, 1, 80, 80])
img.dtype torch.float32
img.device cpu
MINIBATCH: img.shapes seg.shape labels.shape, index.shape -> torch.Size([8, 1, 80, 80]) torch.Size([8, 1, 80, 80]) torch.Size([8]) torch.Size([8])
MINIBATCH: min and max of minibatch tensor(0.) tensor(1.)


In [24]:
vae = CompositionalVae(params)
optimizer = instantiate_optimizer(model=vae, dict_params_optimizer=params["optimizer"])

In [50]:
for epoch in range(1):
    train_metrics = process_one_epoch(model=vae, 
                                      dataloader=test_loader, 
                                      optimizer=optimizer, 
                                      verbose=(epoch == 0),
                                      weight_clipper=None)

0
i =   0 train_loss=18.35329
i =   1 train_loss=17.33237
i =   2 train_loss=20.44165


In [ ]:
img_to_segment = train_loader.img[0, :, 1060:1360, 2100:2400]
plt.imshow(img_to_segment[0])

In [ ]:
ix = 1240-160
iy= 2140
print(ix)
img_to_segment = train_loader.img[0, :, ix:ix+2*(80), iy:iy+4*(80)]
plt.imshow(img_to_segment[0])

In [ ]:

img_to_segment = train_loader.img[0, :, 940:1240, 2140:2440]
plt.imshow(img_to_segment[0])

In [ ]:
Entropy(\pi)

In [ ]:
# params

In [ ]:
image_size = params['input_image']['size_raw_image']
b, c, w, h = 8, params['input_image']['ch_in'], image_size, image_size
imgs_in = torch.sigmoid(torch.randn(b,c,w, h))



In [ ]:
show_batch(imgs_in)

In [ ]:
#from MODULES.utilities_ml import SpecialDataSet
#
#train_loader = SpecialDataSet(img=imgs_in,
#                              store_in_cuda=False,
#                              shuffle=True,
#                              drop_last=True,
#                              batch_size=2)
#optimizer = instantiate_optimizer(model=vae, dict_params_optimizer=params["optimizer"])
#epoch_restart = -1
#history_dict = {}
#min_test_loss = 99999999
#    
#for delta_epoch in range(1, 5):
#    epoch = delta_epoch+epoch_restart
#    with torch.autograd.set_detect_anomaly(True):
#        train_metrics = process_one_epoch(model=vae, 
#                                          dataloader=train_loader, 
#                                          optimizer=optimizer, 
#                                          verbose=True)
#        print(train_metrics.pretty_print(epoch))

In [ ]:
vae.inference_and_generator.similarity_kernel_dpp.get_l_w()

In [ ]:
print(generated._fields)
print(generated.inference._fields)
print(generated.inference.sample_c_map.shape)
show_batch(generated.inference.sample_c_map.float())

In [ ]:
#generated.inference.prob_map[0,0]

In [ ]:
show_batch(generated.inference.prob_map)

In [ ]:
from MODULES.utilities_visualization import plot_reconstruction_and_inference, plot_segmentation

plot_reconstruction_and_inference(generated, epoch=0, prefix="gen_")

In [ ]:
vae.prob_corr_factor = 0.0
output = vae.forward(imgs_in, draw_image=True, draw_boxes=True, verbose=False)

In [ ]:
print(torch.max(output.inference.prob_map))
show_batch(output.inference.prob_map)

In [ ]:
c_map_2 = (torch.rand_like(output.inference.prob_map) < output.inference.prob_map).float()
show_batch(c_map_2)

In [ ]:
show_batch(output.inference.sample_c_map)

In [ ]:
show_batch(output.inference.sample_c_map)

In [ ]:
print(output.inference.bb_few.bh[..., 0])
print(vae.inference_and_generator.size_min)
print(vae.inference_and_generator.size_max)

In [ ]:
plot_reconstruction_and_inference(output, epoch=0, prefix="rec_")

In [ ]:
vae